# Abundances

`synthesizer` can be used to generate abundance patterns, in total and for gas and dust individually, for a given metallicity, alpha ehancement, and arbitrary element scalings.

At present this functionality is only utilised when creating cloudy input scripts. These scripts are used to calculate nebular line and continuum emission for a given incident spectral energy distribution and gas abundance pattern.

In [ ]:
import numpy as np
from synthesizer.abundances import (
    Abundances,
    abundance_scalings,
    solar_abundance_patterns,
    depletion_models,
    plot_abundance_pattern,
    plot_multiple_abundance_patterns,
)

by default initialising `Abundances` creates a solar abundance pattern with no depletion. The default solar abundance pattern is Asplund et al. (2009), though this can be changed if desired. 

In [ ]:
a0 = Abundances()

like most `synthesizer` objects we can explore the important attributes of an object by using `print()`:

In [ ]:
print(a0)

You can access the logarithmic abundances ($\log_{10}(N_X/N_H)$) of an element like this:

In [ ]:
print(f"log10(O/H): {a0.total['O']:.2f}")
print(f"log10(O/H): {a0['O']:.2f}")

### Solar abundance pattern

As noted, there are several Solar abundance patterns built into `synthesizer`. These can be accessed from `synthesizer.abundances.SolarAbundances`:

In [ ]:
solar_abundance_patterns.available_patterns

In [ ]:
solar = solar_abundance_patterns.Asplund2009
solar.ads
solar.abundance

In [ ]:
a1 = Abundances(solar=solar_abundance_patterns.Gutkin2016)
print(a1)

Solar abundance classes can also be called using a string representation of the name.

In [ ]:
a1 = Abundances(solar='Gutkin2016')
print(a1)

### Metallicity

We can specify a different metallicity. By default abundances are scaled from the Solar abundances provided through an optional argument (default Asplund et a. 2009). However, as we will see later, it is possible to set a different $\alpha$-enhancement or set arbitrary element scalings.

In [ ]:
a2 = Abundances(metallicity=0.01)
print(a2)

## $\alpha$-enhancement

We can also generate abundance patterns assuming different $\alpha$-enhancements. In this case it is necessary to re-scale the non-$\alpha$ elements to recover the input metallicity.

In [ ]:
a3 = Abundances(alpha=0.6)
print(a3)

We can print a relative solar abundance like this:

In [ ]:
print(f"[O/Fe] = {a3.solar_relative_abundance('O', ref_element='Fe'):.2f}")
print(f"[O/Fe] = {a3['[O/Fe]']:.2f}")
print(f"[B/Fe] = {a3['[B/Fe]']:.2f}")

That, is, as expected given that we set $\alpha=0.6$.

### Depletion

To account for metals being locked up in dust, we can also specify a depletion pattern. It is possible to either provide a dictionary of values or specify one of the in-built patterns. 

In [ ]:
# assume 99% of Carbon and Iron are depleted on to dust
depletion = {'C': 0.99, 'Fe': 0.99}

# calculate the abundance patterns, now included gas and dust separately
a4 = Abundances(metallicity=0.01, depletion=depletion)
print(a4)

In [ ]:
a5 = Abundances(metallicity=0.0156, solar=solar_abundance_patterns.Gutkin2016, depletion_model=depletion_models.Gutkin2016)
print(a5)

In [ ]:
a6 = Abundances(metallicity=0.0156, solar=solar_abundance_patterns.Gutkin2016, depletion_model=depletion_models.CloudyClassic)
print(a6)

In [ ]:
a7 = Abundances(metallicity=0.0156, solar=solar_abundance_patterns.Gutkin2016, depletion_model=depletion_models.Jenkins2009)
print(a7)

When specifying an in-built pattern it's also possible to specify an optional scaling parameter depending on the particular model.

Below we explore the effect of $F_{*}$ on the depletion factors for N, O, and S. 

In [ ]:

for element in ['N','O', 'S']:
    print(element, '-'*5)
    for fstar in [0.0, 0.5, 1.0]:
        depletion = depletion_models.Jenkins2009(fstar).depletion[element]
        print(f"{fstar} {depletion:.2f} {np.log10(depletion):.2f}")

Next, we explore the effect of $F_{*}$ on the dust mass fraction and dust-to-metal ratio ($\xi_{d}$):

In [ ]:
for fstar in [0.0, 0.1, 0.25, 0.5, 1.0]:
    a = Abundances(metallicity=0.0156, solar=solar_abundance_patterns.Gutkin2016, depletion_model=depletion_models.Jenkins2009, depletion_scale=fstar)
    print(fstar, f'{a.dust_mass_fraction:.4f}', f'{a.dust_to_metal_ratio:.2f}')


When the depletion is applied the total, gas, and dust abundance patterns are provided e.g.

In [ ]:
print(f'log10(C_total/H) : {a7.total["C"]:.2f}')
print(f'log10(C_gas/H) : {a7.gas["C"]:.2f}')
print(f'log10(C_dust/H) : {a7.dust["C"]:.2f}')

### Arbitrary element scaling

We can also change the abundance of any specific element (or set of elements), with the abundances of other elements rescaled self-consistently to yield the correct metallicity. 

If the abundance is a float it is the logarithmic abundance ($\log_{10}(X/H)$) while if it is a string it is one of the in-built functions that scale the abundance with metallicity (e.g. the model proposed by Dopita et al. 2006). Note, combining this with a non-zero `alpha` can lead to a mild inconsistency.

Using a float:

In [ ]:
a8 = Abundances(metallicity=0.0134, abundances={"N": -4.5})
print(a8)

Using a specified function:

In [ ]:
a9 = Abundances(metallicity=0.0134, abundances={"N": "Dopita2006"})
print(a9)

We can also access the scaling functions directly:

In [ ]:
abundance_scalings.available_scalings

In [ ]:
abundance_scalings.Dopita2006.available_elements

In [ ]:
abundance_scalings.Dopita2006.N(0.016)

These functions also include useful meta data:

In [ ]:
print(abundance_scalings.Dopita2006.ads)
print(abundance_scalings.Dopita2006.doi)

## Plots

There are also a helper functions for plotting one or more abundance patterns, here we plot two abundance patterns with different alpha abundances:

In [ ]:
plot_multiple_abundance_patterns(
    [a2, a3],
    labels=[r"Z=0.01", r"Z=0.01; \alpha = 0.6"],
    show=True,
    ylim=[-7.0, -3.0],
)

We can plot the abundance pattern of each component:

In [ ]:
plot_abundance_pattern(
    a7, show=True, ylim=[-7.0, -3.0], components=["total", "gas", "dust"]
)